In [1]:
### 2. C3D+ucf_model_training_multigpu
# Author : @leopauly | cnlp@leeds.ac.uk <br>
# Description : Training the C3D model using UCF 101 action recognition dataset

print('Started running the program..!',flush=True)
## Imports
from keras.models import Sequential
import random
import numpy as np
from PIL import Image
from os import listdir
from scipy.ndimage import imread
import keras
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras import backend as K
import datetime
import time
import os 
from datetime import timedelta

# Custom scripts
import lscript as lsp
import modelling as md
#from DataSet.DataSet import DataSet
import dataset as dset
import ucf101_dataset as ucf

print('Loaded libraries...!!',flush=True)

Started running the program..!


Using TensorFlow backend.


Loaded libraries...!!


In [2]:
height=112 
width=112 
channel=3
cluster_length=16
nb_classes=101

lr_rate=.001
next_batch_start=0
batch_size=8
batch_size_test=8
total_train_videos=9991
memory_batch_size_train=50
memory_batch_size_test=300
iterations= 10 # (int(total_train_videos/memory_batch_size_train)) #10001
custom_global_step=0
LOG_DIR='/nobackup/leopauly/logdir'

print('Finished defining variables..!!',flush=True)

Finished defining variables..!!


In [3]:
## Finding how many devices are available
gpus = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']
num_gpus = len(gpus)
print("GPU nodes found: " + str(num_gpus),flush=True)
for i in range(num_gpus):
    print('Avaialble gpu:',str(gpus[i]),flush=True)


## Finding how many CPUs are available
cpus = [x.name for x in device_lib.list_local_devices() if x.device_type == 'CPU']
num_cpus = len(cpus)
print("CPU nodes found: " + str(num_cpus),flush=True)
for i in range(num_cpus):
    print('Avaialble cpu:',str(cpus[i]),flush=True)


## Defining placeholders in tf for images and targets
x_image = tf.placeholder(tf.float32, [None, 16,height,width,channel]) 
y_true = tf.placeholder(tf.float32, [None, nb_classes])
y_true_cls = tf.placeholder(tf.int64, [None])

print('Finished obtaining gpu details and place holders',flush=True)

GPU nodes found: 1
Avaialble gpu: /gpu:0
CPU nodes found: 1
Avaialble cpu: /cpu:0
Finished obtaining gpu details and place holders


In [4]:
#### Creating the model and defining the function to parallelise data
# Define the network in a model function, to make parallelisation across GPUs easier.
def model(x_image_, y_true_):
    ''' Expecting the following parameters, in batches:
        x_image_ - x_image batch
        y_true_ - y_true batch
    '''

    model_keras = md.C3D_ucf101_training_model_tf(summary=True)
    #model_keras = modelC3D(cluster_length, height, width, channel,summary=False, load_weights=True)
    out=model_keras(x_image_)
    print(out,flush=True)
    
    y_pred = tf.nn.softmax(out)
    y_pred_cls = tf.argmax(out, dimension=1)
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true_,logits=out))
    
    # Outputs to be returned to CPU
    return y_pred, y_pred_cls, loss


def make_parallel(fn, **kwargs):
    in_splits = {}
    for k, v in kwargs.items():
        in_splits[k] = tf.split(v, num_gpus)
    # An array for every aggregated output
    y_pred_split, y_pred_cls_split, cost_split = [], [], []
    for i in range(num_gpus):
        with tf.device(tf.DeviceSpec(device_type="GPU", device_index=i)):
            with tf.variable_scope(tf.get_variable_scope(), reuse=i > 0):
                y_pred_, y_pred_cls_, cost_,  = fn(**{k : v[i] for k, v in in_splits.items()})
                # Adding the output from each device.
                y_pred_split.append(y_pred_)
                y_pred_cls_split.append(y_pred_cls_)
                cost_split.append(cost_)
                #fv_split.append(fv_)
    # Aggregating and returning outputs. tf.concat for multi-dimensional arrays; tf.stack if single values.
    return tf.concat(y_pred_split, axis=0), tf.concat(y_pred_cls_split, axis=0),tf.stack(cost_split, axis=0)

print('Finished defining special functions',flush=True)
  
if num_gpus > 0:
    # There is significant latency for CPU<->GPU copying of shared variables.
    # We want the best balance between speedup and minimal latency.
    y_pred, y_pred_cls, cost = make_parallel(model, x_image_=x_image, y_true_=y_true)
else:
    # CPU-only version
    y_pred, y_pred_cls, cost = model(x_image_=x_image, y_true_=y_true)

Finished defining special functions


/home/ufaserv1_f/cnlp/Seeing_to_Learn/Observation-Learning/Stage_5_New_loss_function/modelling.py:392: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", name="conv1", input_shape=(16, 112, ..., padding="same")`
  input_shape=input_shape))
/home/ufaserv1_f/cnlp/Seeing_to_Learn/Observation-Learning/Stage_5_New_loss_function/modelling.py:394: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  border_mode='valid', name='pool1'))
/home/ufaserv1_f/cnlp/Seeing_to_Learn/Observation-Learning/Stage_5_New_loss_function/modelling.py:397: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", name="conv2", padding="same")`
  border_mode='same', name='conv2'))
/home/ufaserv1_f/cnlp/Seeing_to_Learn/Observation-Learning/Stage_5_New_loss_function/modelling.py:399: UserWarning: Update your `MaxPooling3D` ca

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 4, 14, 14, 256)    0         
__________

In [ ]:
# Optimisation calculated on CPU on aggregated results.
# NEED the colocate_gradients_with_ops flag TRUE to get the gradient ops to run on same device as original op!
optimizer = tf.train.AdagradOptimizer(learning_rate=2e-4).minimize(cost, colocate_gradients_with_ops=True)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Miscellenious items finished..!!',flush=True)

In [ ]:
#### Training & testing
def testing(iterations,loops):
    #print(test_images.shape)
    #print(test_labels_cls)
    test_score=0
    for j in range(int(memory_batch_size_test/batch_size_test)-1):
        test_score_ = sess.run([accuracy], feed_dict={x_image:test_images[(batch_size_test*j):(batch_size_test*(j+1))],y_true_cls:test_labels_cls[(batch_size_test*j):(batch_size_test*(j+1))],K.learning_phase(): 0 })
        #print('returned value',test_score_)
        test_score=test_score+sum(test_score_)
    print('Test accuracy after iteration:',iterations,',loop:',loops,'is:',test_score/(j+1),flush=True)

In [ ]:
## Loading Testing data
test_images, test_labels_cls, next_batch_start, _ = ucf.read_vid_and_label('./UCF101_data_preparation/test.list',memory_batch_size_test,-1,16,112,normalisation=True)
test_labels=keras.utils.to_categorical(test_labels_cls, num_classes=nb_classes)
print('testing data loaded',flush=True)

In [ ]:
#### Start the session with logging placement.
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)

testing(0,0)

In [ ]:
#### Start the session with logging placement.
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)
saver = tf.train.Saver() 
print ('started the session...!!',flush=True)

## Loading Training data
start_time = time.time()
train_images, train_labels_cls, next_batch_start, _ = ucf. read_vid_and_label('./UCF101_data_preparation/train.list',memory_batch_size_train,-1,cluster_length,112,normalisation=True)
end_time = time.time()
time_dif = end_time - start_time
print("Time usage for loading training dataset: " + str(timedelta(seconds=int(round(time_dif)))),flush=True)
train_labels=keras.utils.to_categorical(train_labels_cls, num_classes=nb_classes)

## Loading Testing data
test_images, test_labels_cls, next_batch_start, _ = ucf.read_vid_and_label('./UCF101_data_preparation/test.list',memory_batch_size_test,-1,16,112,normalisation=True)
test_labels=keras.utils.to_categorical(test_labels_cls, num_classes=nb_classes)
print('testing data loaded',flush=True)

for i in range(0,(iterations*10)):
    print('started iteration:',i,flush=True)
    
    for j in range (int(memory_batch_size_train/batch_size)-1):    
        print ('This is epoch:',j,'going to be trained',flush=True)
        output_value = sess.run([optimizer], feed_dict={x_image:train_images[(batch_size*j):(batch_size*(j+1))],y_true:train_labels[(batch_size*j):(batch_size*(j+1))],K.learning_phase(): 1 })   
        print ('This is epoch:',j,'trained',flush=True)
        if (int(j%300)==0):
            testing(i,j)
            #print('tested')
    testing(i,j)
    saver.save(sess, os.path.join(LOG_DIR, "activity_model_1.ckpt"), global_step=custom_global_step)
    custom_global_step=custom_global_step+1   
    print('Model saved after iteration:',i,flush=True)


sess.close()